In [1]:
!pip install transformers==4.29.2
!pip install accelerate==0.19.0
!pip install torch==2.0.0
!pip install einops==0.6.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 7.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 108.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB

In [ ]:
!nvidia-smi

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
import torch

model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b", torch_dtype=torch.float16).cuda()

# the fast tokenizer currently does not work correctly
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b", use_fast=False)

In [4]:
instruction = "How do you pick a co founder?"
prompt = f"""Answer the following Question : {instruction}

Response: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

set_seed(32)
generated_ids = model.generate(input_ids, do_sample=True,max_new_tokens=1024)

tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

['Answer the following Question : How do you pick a co founder?\n\nResponse:  http://answertabs.com/answers/search.aspx?QuestionId=8983&PostId=135949\nThere are quite a few answers here, so it may be good to break it down for those of us who are new to this.\n\nFrom the post: "I want to share my co-founder and make sure our interests align and complement each other because we are not in competition."\n\nThen you should only consider two options.\n\nOption 1\nYou should pick your co founder based only on the following:\n1\\. Quality of relationship.\n2\\. Passion\n\n2\\. A very rare talent and rare market need that will require a highly motivated cofounder.\nThe "pursuit of happiness" in this case is more the "what is in it for me" side that really matters.\n\nWe don\'t need to agree on what we want to do with this company. As long as it makes sense to us then it\'s worth it to give our best and see if the other guy will agree....\n\nSo I got this idea.\nWe create a social media where p

In [ ]:
import json

# Read the JSON data from file
with open('questions.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Remove data points with the same query_string
unique_entries = []
query_strings_seen = set()

for entry in json_data:
    query_string = entry['query_string']

    if query_string not in query_strings_seen:
        unique_entries.append(entry)
        query_strings_seen.add(query_string)

# Open the file to append the data
with open('opt-1.3b.txt', 'a', encoding='utf-8') as output_file:
    ct = 1

    # Iterate over the unique data entries and extract relevant information
    for entry in unique_entries:
        query_string = "Answer the following question : "
        query_string += entry['query_string']

        # # Generate text
        # res = generate_text(
        #     query_string,
        #     min_new_tokens=2,
        #     max_new_tokens=1024,
        #     do_sample=False,
        #     num_beams=1,
        #     temperature=float(0.0),
        #     repetition_penalty=float(1.2),
        #     renormalize_logits=True
        # )

        input_ids = tokenizer(query_string, return_tensors="pt").input_ids.cuda()

        set_seed(32)
        generated_ids = model.generate(input_ids, do_sample=True,max_new_tokens=1024)

        generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        # generated_text = res[0]["generated_text"]  # Assuming generate_text() returns the generated text

        # Write query_string and generated_text to the file
        output_file.write(f'["{query_string}", "{generated_text}"]\n')
        print(ct)
        ct += 1
        print(generated_text)
